# Entwicklung der Stromproduktion in der Schweiz: Solarenergie vs. andere Quellen
Dieses Notebook analysiert die Entwicklung der Stromproduktion aus Solarenergie im Vergleich zu anderen Energiequellen in der Schweiz in den letzten 10 Jahren.

## Datenimport und Vorbereitung
Die Daten werden aus einer CSV-Datei geladen, bereinigt und für die Visualisierung vorbereitet.

In [ ]:
# Bibliotheken importieren
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# CSV-Datei laden (Trennzeichen: Semikolon)
file_path_energy_data = '../data/energy_sources/elektrizitaetbilanz_jahreswerte.csv'
energy_data = pd.read_csv(file_path_energy_data, delimiter=';')

# Nur relevante Spalten behalten: Jahr + alle Spalten, die mit 'Erzeugung_' beginnen
columns_to_keep = ['Jahr'] + [col for col in energy_data.columns if col.startswith('Erzeugung_')]
energy_data = energy_data[columns_to_keep]

# Aggregierte Spalten für Wasserkraft und andere erneuerbare Energien berechnen
energy_data['Wasserkraft'] = (
    energy_data['Erzeugung_laufwerk_GWh'] + energy_data['Erzeugung_speicherwerk_GWh']
)
energy_data['Andere_erneuerbare_Energien_GWh'] = (
    energy_data['Erzeugung_holz_GWh'] +
    energy_data['Erzeugung_biogas_GWh'] +
    energy_data['Erzeugung_wind_GWh']
)

# Daten ab 2015 filtern (letzte 10 Jahre)
energy_data = energy_data[energy_data['Jahr'] >= 2015].copy()

# Jahr als String für bessere Achsenbeschriftung
energy_data['Jahr'] = energy_data['Jahr'].astype(str)

# Spalten für die Visualisierung umbenennen
energy_data.rename(columns={
    'Erzeugung_kernkraftwerk_GWh': 'Kernkraft',
    'Erzeugung_andere_total_GWh': 'Thermische Kraft',
    'Erzeugung_photovoltaik_GWh': 'Photovoltaik',
    'Andere_erneuerbare_Energien_GWh': 'Andere erneuerbare Energien'
}, inplace=True)

# Spalten für die Visualisierung auswählen
columns_to_melt = [
    'Wasserkraft',
    'Kernkraft',
    'Thermische Kraft',
    'Photovoltaik',
    'Andere erneuerbare Energien'
]

# Daten für die Visualisierung transformieren (long-Format)
energy_data_melted = energy_data.melt(
    id_vars=['Jahr'],
    value_vars=columns_to_melt,
    var_name='Energiequelle',
    value_name='Wert'
)

## Entwicklung der Stromproduktion nach Energiequelle (Flächenchart)
Das folgende Flächendiagramm zeigt die Entwicklung der Stromproduktion nach Energiequelle in der Schweiz seit 2015.

In [ ]:
# Flächenchart für die Entwicklung der Stromproduktion erstellen
fig = px.area(
    energy_data_melted,
    x='Jahr',
    y='Wert',
    color='Energiequelle',
    title='Stromproduktion nach Energiequelle in der Schweiz',
    labels={
        'Wert': 'Stromproduktion (GWh)',
        'Jahr': 'Jahr',
        'Energiequelle': 'Energiequelle'
    },
    color_discrete_sequence=['#f6c90e', '#3c91e6', '#52de97', '#fa744f', '#8a6ded']  # Konsistente Farben
)

# Layout anpassen: Titel zentrieren, Legende unten, Schriftgrösse erhöhen
fig.update_layout(
    title=dict(x=0.5),
    yaxis_title='Stromproduktion (GWh)',
    legend_title='Energiequelle',
    font=dict(size=12),
    margin=dict(t=50, b=50, l=50, r=50),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.2,
        xanchor="center",
        x=0.5
    )
)

# Diagramm als HTML speichern und anzeigen
fig.write_html('../docs/assets/diagramme/solar_vs_other_energy_sources.html')
fig.show()

## Prozentuale Verteilung der Stromproduktion (Kuchendiagramm mit Slider)
Das Kuchendiagramm zeigt die prozentuale Verteilung der Stromproduktion nach Energiequelle für ein ausgewähltes Jahr.

In [ ]:
# Farben für die Energiequellen definieren (konsistent mit Flächenchart)
farben = {
    "Photovoltaik": "#f6c90e",
    "Kernkraft": "#3c91e6",
    "Wasserkraft": "#52de97",
    "Thermische Kraft": "#fa744f",
    "Andere erneuerbare Energien": "#8a6ded"
}

# Energiequellen und verfügbare Jahre extrahieren
quellen = list(farben.keys())
years = energy_data["Jahr"].tolist()

# Werte für das erste Jahr extrahieren
werte_0 = energy_data.loc[energy_data["Jahr"] == years[0], quellen].values.flatten()

# Kuchendiagramm für das erste Jahr erstellen
fig = go.Figure()
fig.add_trace(go.Pie(
    labels=quellen,
    values=werte_0,
    name=str(years[0]),
    hole=0.4,  # Donut-Chart
    sort=False,
    rotation=120,
    marker=dict(
        colors=[farben[q] for q in quellen],
        line=dict(color="#ffffff", width=2)
    ),
    textinfo="percent+label",
    hovertemplate="%{label}: %{value:.0f} GWh<br>%{percent:.1%}<extra></extra>",
    insidetextfont=dict(size=14),
    texttemplate="%{label}: %{percent:.1%}"
))

# Slider-Schritte für jedes Jahr vorbereiten
steps = []
for year in years:
    werte = energy_data.loc[energy_data["Jahr"] == year, quellen].values.flatten()
    step = dict(
        method="restyle",
        args=[{
            "values": [werte],
            "marker": [dict(colors=[farben[q] for q in quellen])],
            "sort": [False],
            "rotation": [120]
        }],
        label=str(year)
    )
    steps.append(step)

# Slider-Layout definieren
sliders = [dict(
    active=0,
    currentvalue={"prefix": "Jahr: ", "font": {"size": 16}},
    pad={"t": 50},
    steps=steps
)]

# Layout des Kuchendiagramms anpassen
fig.update_layout(
    title=dict(
        text="Stromproduktion Schweiz – Prozentuale Verteilung nach Energiequelle (GWh)",
        x=0.5,
        font=dict(size=20)
    ),
    sliders=sliders,
    font=dict(family="Arial", size=14),
    margin=dict(t=100, b=50, l=50, r=50),
    showlegend=False
)

# Diagramm als HTML speichern und anzeigen
fig.write_html("../docs/assets/diagramme/stromproduktion_kuchendiagramm_slider.html")
fig.show()

## Interaktive Treemap: Stromproduktion nach Energiequelle und Jahr
Die folgende Treemap visualisiert die Verteilung der Stromproduktion nach Energiequelle für jedes Jahr. Über den Slider kann das Jahr ausgewählt werden.

In [ ]:
import plotly.graph_objects as go

# Für die Treemap: Daten für jedes Jahr vorbereiten
frames = []
for year in years:
    values = energy_data.loc[energy_data["Jahr"] == year, quellen].values.flatten()
    frames.append(dict(
        data=[go.Treemap(
            labels=quellen,
            parents=[""] * len(quellen),
            values=values,
            marker=dict(colors=[farben[q] for q in quellen]),
            textinfo="label+percent entry+value",
            texttemplate="%{label}<br>%{percentEntry:.1%}<br>%{value:.0f} GWh",
            hovertemplate="%{label}: %{value:.0f} GWh<br>%{percentEntry:.1%}<extra></extra>"
        )],
        name=str(year)
    ))

# Initiale Treemap für das erste Jahr
fig_treemap = go.Figure(
    data=[go.Treemap(
        labels=quellen,
        parents=[""] * len(quellen),
        values=energy_data.loc[energy_data["Jahr"] == years[0], quellen].values.flatten(),
        marker=dict(colors=[farben[q] for q in quellen]),
        textinfo="label+percent entry+value",
        texttemplate="%{label}<br>%{percentEntry:.1%}<br>%{value:.0f} GWh",
        hovertemplate="%{label}: %{value:.0f} GWh<br>%{percentEntry:.1%}<extra></extra>"
    )],
    frames=frames
)

# Slider für die Jahre konfigurieren
sliders_dict = {
    "active": 0,
    "steps": [
        {
            "args": [[str(year)], {"frame": {"duration": 0, "redraw": True}, "mode": "immediate", "transition": {"duration": 0}}],
            "label": str(year),
            "method": "animate"
        }
        for year in years
    ],
    "x": 0.5,
    "xanchor": "center",
    "y": -0.1,
    "yanchor": "top",
    "currentvalue": {"prefix": "Jahr: ", "font": {"size": 16}},
    "pad": {"t": 50}
}

# Layout der Treemap anpassen
fig_treemap.update_layout(
    title=dict(
        text="Treemap der Stromproduktion nach Energiequelle (GWh)",
        x=0.5,
        font=dict(size=20)
    ),
    margin=dict(t=100, b=50, l=50, r=50),
    sliders=[sliders_dict],
    font=dict(family="Arial", size=14),
    updatemenus=[{
        "type": "buttons",
        "showactive": False,
        "buttons": [
            {
                "label": "Play",
                "method": "animate",
                "args": [None, {"frame": {"duration": 800, "redraw": True}, "fromcurrent": True}]
            },
            {
                "label": "Pause",
                "method": "animate",
                "args": [[None], {"frame": {"duration": 0, "redraw": False}, "mode": "immediate", "transition": {"duration": 0}}]
            }
        ],
        "direction": "left",
        "x": 0.5,
        "xanchor": "center",
        "y": -0.18,
        "yanchor": "top"
    }]
)

# Diagramm als HTML speichern und anzeigen
fig_treemap.write_html("../docs/assets/diagramme/stromproduktion_treemap_slider.html")
fig_treemap.show()